In [1]:
import argparse
import json
import os
from pathlib import Path
from typing import Sequence, Optional, Dict, Tuple

import matplotlib.pyplot as plt
import numpy as np
from numpy import ndarray
from tqdm import tqdm

from keras_models import generate_ncp_model
import tensorflow as tf

from typing import Tuple, Optional

import PIL.Image
import numpy as np
import tensorflow as tf
from numpy import ndarray
from tensorflow.python.keras.models import Functional


2024-10-04 14:54:45.294673: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
!export TF_CPP_MIN_LOG_LEVEL=2

In [3]:
def generate_hidden_list(model: Functional, return_numpy: bool = True):
    print(model.input_shape)
    constructor = np.zeros if return_numpy else tf.zeros
    hiddens = []
    if len(model.input_shape)==1:
        lool = model.input_shape[0][1:]
    else:
        lool = model.input_shape[1:]
    print(lool)
    for input_shape in lool:  # ignore 1st output, as is this control output
        hidden = []
        for i, shape in enumerate(input_shape):
            if shape is None:
                if i == 0:  # batch dim
                    hidden.append(1)
                    continue
                elif i == 1:  # seq len dim
                    hidden.append(0)
                    continue
                else:
                    print("Unable to infer hidden state shape. Leaving as none")
            hidden.append(shape)
        hiddens.append(constructor(hidden))
    return hiddens

In [4]:
def load_image(img_path: str, img_shape: Tuple[int, int, int], reverse_channels: bool = True) -> Optional[ndarray]:
    img = PIL.Image.open(img_path)
    if img is not None:
        # image shape is height, width, PIL takes width, height
        resized = img.resize(img_shape[:2][::-1], PIL.Image.BILINEAR)
        img_numpy = tf.keras.preprocessing.image.img_to_array(resized).astype(np.uint8)
        if reverse_channels:
            # reverse channels of image to match training
            img_numpy = img_numpy[..., ::-1]

        # add batch dim
        img_numpy = np.expand_dims(img_numpy, axis=0)
        return img_numpy
    else:
        return None


def image_dir_generator(data_path: str, image_shape: Tuple[int, int, int], reverse_channels: bool = False):
    
    contents = os.listdir(data_path)
    contents = [os.path.join(data_path, c) for c in contents if 'png' in c]
    contents.sort()
    for path in contents:
        img = load_image(img_path=path, img_shape=image_shape, reverse_channels=reverse_channels)
        if img is not None:
            yield img

In [5]:
reverse_channels = False
sequence_path = "../../fly_to_target_dataset/dataset/1"

IMAGE_SHAPE = (144, 256, 3)
IMAGE_SHAPE_CV = (IMAGE_SHAPE[1], IMAGE_SHAPE[0])

batch_size = None
seq_len = 64
augmentation_params = None
single_step = True
no_norm_layer = False
DROPOUT = 0.1

DEFAULT_NCP_SEED = 22222

mymodel = generate_ncp_model(seq_len, IMAGE_SHAPE, augmentation_params, batch_size, DEFAULT_NCP_SEED, single_step, no_norm_layer)

mymodel.load_weights('../saved_models/retrain_difftraj_wscheduler0.85_seed22222_lr0.001_trainloss0.00016_valloss0.13141_diffcoreset900.h5')

hiddens = generate_hidden_list(model=mymodel, return_numpy=True)
print("hidden", hiddens)

all_hiddens = []  # list of list of arrays with shape num_timesteps x num_hiddens x hidden_dim
for i, img in tqdm(enumerate(image_dir_generator(sequence_path, IMAGE_SHAPE, reverse_channels))):
    all_hiddens.append(hiddens)
    out = mymodel.predict([img, *hiddens])
    hiddens = out[1:]  # list num_hidden long, each el is hidden_dim,

# print("all_hiddens", all_hiddens)
# flatten batch dim
all_hiddens = [[np.squeeze(hid, axis=0) for hid in step_hid] for step_hid in all_hiddens]
# print("all_hiddens", all_hiddens)
# crete list with same shape as hidden vectors where contents are lipschitz values of each dimension
lip = [np.zeros_like(h) for h in all_hiddens[0]]
for i in range(len(all_hiddens) - 1):
    current_hiddens = all_hiddens[i]
    next_hiddens = all_hiddens[i + 1]
    diff = [np.abs(n - c) for n, c in zip(next_hiddens, current_hiddens)]
    lip = [np.maximum(l, d) for l, d in zip(lip, diff)]
print(lip)


2024-10-04 14:54:53.882479: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-10-04 14:54:53.883183: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-10-04 14:54:53.906638: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-10-04 14:54:53.906659: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: iras-hub
2024-10-04 14:54:53.906666: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: iras-hub
2024-10-04 14:54:53.906756: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 550.54.14
2024-10-04 14:54:53.906773: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 550.54.14
2024-10-04 14:54:53.906779: I tensorflow/stream_executor/cu

[(None, 144, 256, 3), (None, 34)]
[(None, 34)]
hidden [array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])]


0it [00:00, ?it/s]2024-10-04 14:54:54.475982: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-10-04 14:54:54.492426: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3000260000 Hz
832it [00:28, 29.05it/s]

[array([0.10736777, 0.09390518, 0.51793343, 0.19179153, 0.35834986,
       0.49816179, 0.52820468, 0.19368741, 0.07618143, 0.04062793,
       0.28867343, 0.10671288, 0.06689268, 1.25927126, 0.59002292,
       0.07122275, 0.04867452, 0.3279714 , 0.39318371, 0.93345183,
       0.46152639, 0.35753256, 0.62972325, 0.26788479, 0.93245256,
       0.74350893, 0.78686345, 0.99237657, 0.93485636, 0.24223337,
       0.10485464, 0.14562921, 0.10232146, 0.65539438])]
